##  Using a list of all the valid DOIs, we will query the CrossRef API to acquire the lists of authors for each refernence and the publication date.

### then use these to generate an count of the number of authors 


#### this work could be added to the end of one of the other dataframes, but this appraoch should generate a cleaner set of nodes

#### We might also wish to generate a list of all of the unique author (node) names and see if we can identifiy variations of the names for the same individual

</br>

In [45]:
import pandas as pd
import itertools
from habanero import Crossref, cn, counts, exceptions
import time

import hvplot.pandas
import holoviews as hv

In [46]:
df1 = pd.read_csv('./author_networks/all_dois.csv', header=None)
df1.shape

(2366, 1)

In [47]:
df1.head()

,0
0,10.1111/j.1460-9592.2011.03591.x
1,10.1016/j.neurobiolaging.2011.05.018
2,10.1037/a0024992
3,10.1016/j.neuroimage.2011.09.010
4,10.1002/hbm.21402


In [48]:
cr = Crossref()

In [49]:
# set a mailto address
Crossref(mailto = "laurence.brown@it.ox.ac.uk")

< Crossref 
URL: https://api.crossref.org
KEY: None
MAILTO: laurence.brown@it.ox.ac.uk
>

In [50]:
df1.iloc[1]

0    10.1016/j.neurobiolaging.2011.05.018
Name: 1, dtype: object

In [81]:
cr.works(ids=df1.iloc[1])['message']

{'indexed': {'date-parts': [[2019, 8, 26]],
  'date-time': '2019-08-26T09:43:19Z',
  'timestamp': 1566812599549},
 'reference-count': 55,
 'publisher': 'Elsevier BV',
 'issue': '4',
 'license': [{'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/',
   'start': {'date-parts': [[2012, 4, 1]],
    'date-time': '2012-04-01T00:00:00Z',
    'timestamp': 1333238400000},
   'delay-in-days': 0,
   'content-version': 'tdm'}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['Neurobiology of Aging'],
 'published-print': {'date-parts': [[2012, 4]]},
 'DOI': '10.1016/j.neurobiolaging.2011.05.018',
 'type': 'journal-article',
 'created': {'date-parts': [[2011, 7, 25]],
  'date-time': '2011-07-25T13:24:12Z',
  'timestamp': 1311600252000},
 'page': '825.e25-825.e36',
 'source': 'Crossref',
 'is-referenced-by-count': 81,
 'title': ["Structural MRI changes detectable up to ten years before clinical Alzheimer's disease"],
 'prefix': '10.1016',
 'volume': '3

In [52]:
def DoiToRecord(df):
    try:
        search = cr.works(ids=df)
        time.sleep(0.1)
        record = search ['message']
        return record
    except:
        pass
        

In [53]:
records = df1.apply(DoiToRecord, axis=1)

In [54]:
records

0       {'indexed': {'date-parts': [[2019, 2, 16]], 'd...
1       {'indexed': {'date-parts': [[2019, 8, 26]], 'd...
2       {'indexed': {'date-parts': [[2019, 8, 22]], 'd...
3       {'indexed': {'date-parts': [[2019, 10, 6]], 'd...
4       {'indexed': {'date-parts': [[2019, 10, 19]], '...
                              ...                        
2361                                                 None
2362    {'indexed': {'date-parts': [[2019, 2, 15]], 'd...
2363                                                 None
2364                                                 None
2365                                                 None
Length: 2366, dtype: object

In [55]:
missing =df1[records.isna()]
missing#.to_clipboard()

,0
2361,10.3233/jpd-120126
2363,10.3978/j.issn.2225-319X.2014.05.14
2364,10.3978/j.issn.2305-5839.2015.09.12
2365,10.5083/ejcm.20424884.147


In [56]:
records_all =records.dropna()#.append(records_retry)
records_df = pd.DataFrame(pd.io.json.json_normalize(records_all.dropna()))

In [57]:
records_df.to_json('./D2out_2362_CrossRef_records_15Oct19.json')
records_df.shape

(2362, 66)

In [58]:
'''
Manual checking process for the remaining references 
(one dead DOI reported to CrossRef, 4 others correct but not present in CrossRef, found in Dimensions.ai confirmation)¶
read back in from Excel and use new colum 

#checked = pd.read_clipboard()
#checked.to_csv('./checked_for_CrossRef(22jul19).csv')
checked =pd.read_csv('./checked_for_CrossRef(15Aug19).csv', index_col=0)

checked 

records_retry = checked.checked.apply(DoiToRecord)

records_retry

records_df.to_csv('./E2out_2363_CrossRef_records_15Aug19.csv')
records_df.shape'''

"\nManual checking process for the remaining references \n(one dead DOI reported to CrossRef, 4 others correct but not present in CrossRef, found in Dimensions.ai confirmation)¶\nread back in from Excel and use new colum \n\n#checked = pd.read_clipboard()\n#checked.to_csv('./checked_for_CrossRef(22jul19).csv')\nchecked =pd.read_csv('./checked_for_CrossRef(15Aug19).csv', index_col=0)\n\nchecked \n\nrecords_retry = checked.checked.apply(DoiToRecord)\n\nrecords_retry\n\nrecords_df.to_csv('./E2out_2363_CrossRef_records_15Aug19.csv')\nrecords_df.shape"

In [59]:
type(records_df.author[24])

list

In [60]:
# Check that the author column can be split to rows, allowing a simple count
pd.DataFrame(records_df.author[24])

,given,family,sequence,affiliation
0,S J L,Knight,first,[]
1,C,Yau,additional,[]
2,R,Clifford,additional,[]
3,A T,Timbs,additional,[]
4,E,Sadighi Akha,additional,[]
5,H M,Dréau,additional,[]
6,A,Burns,additional,[]
7,C,Ciria,additional,[]
8,D G,Oscier,additional,[]
9,A R,Pettitt,additional,[]


In [61]:
def AuthCount(df):
    try:
        author_count = len(df.author)
    except:
        author_count = None
    return author_count

In [62]:
len(records_df.author[907])

9

In [63]:
DOI_auth_count = records_df.assign(auth_number = records_df.apply(AuthCount, axis=1))
DOI_auth_count.head()

,reference-count,publisher,issue,license,short-container-title,DOI,type,page,source,is-referenced-by-count,...,relation.is-supplemented-by,assertion,update-to,event.name,event.location,relation.has-preprint,event.start.date-parts,event.end.date-parts,relation.has-review,auth_number
0,48,Wiley,5,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,[],10.1111/j.1460-9592.2011.03591.x,journal-article,615-622,Crossref,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,55,Elsevier BV,4,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[Neurobiology of Aging],10.1016/j.neurobiolaging.2011.05.018,journal-article,825.e25-825.e36,Crossref,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
2,0,American Psychological Association (APA),4,NaN,[Emotion],10.1037/a0024992,journal-article,778-784,Crossref,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,20,Elsevier BV,2,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[NeuroImage],10.1016/j.neuroimage.2011.09.010,journal-article,648-651,Crossref,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,41,Wiley,12,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,[Hum. Brain Mapp.],10.1002/hbm.21402,journal-article,2802-2814,Crossref,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0


In [64]:
DOI_auth_count.set_index(pd.DatetimeIndex(DOI_auth_count['created.date-time']),inplace=True)


In [65]:
DOI_auth_count.sort_index(inplace=True)
DOI_auth_count['expanding_median']= DOI_auth_count.auth_number.rolling(window=30).median()

In [66]:
DOI_auth_count.auth_number.describe()

count    2347.000000
mean       17.992331
std        65.458954
min         1.000000
25%         5.000000
50%         9.000000
75%        14.000000
max      2467.000000
Name: auth_number, dtype: float64

In [67]:
records_start =pd.read_csv('./author_networks/dois_Start.csv', header=None, names=['DOI'])
records_start.head()

,DOI
0,10.1111/j.1460-9592.2011.03591.x
1,10.1016/j.neurobiolaging.2011.05.018
2,10.1037/a0024992
3,10.1016/j.neuroimage.2011.09.010
4,10.1002/hbm.21402


In [68]:
records_mid =pd.read_csv('./author_networks/dois_Mid.csv', header=None, names=['DOI'])
records_mid.head()

,DOI
0,10.2217/pgs.13.183
1,10.1038/bmt.2013.178
2,10.1162/jocn_a_00526
3,10.1002/mrm.25015
4,10.1080/15265161.2013.849307


In [69]:
records_end =pd.read_csv('./author_networks/dois_End.csv', header=None, names=['DOI'])
records_end.head()

,DOI
0,10.1378/chest.15-0416
1,10.1016/j.ejrad.2015.06.025
2,10.1016/j.ajo.2015.06.018
3,10.1136/bmjopen-2014-007376
4,10.1016/j.exphem.2015.06.008


In [70]:
DOI_auth_count_start = DOI_auth_count.merge(records_start, on='DOI',how='inner')
DOI_auth_count_start.shape

(780, 68)

In [71]:
DOI_auth_count_mid = DOI_auth_count.merge(records_mid, on='DOI',how='inner')
DOI_auth_count_mid.shape

(782, 68)

In [72]:
DOI_auth_count_end = DOI_auth_count.merge(records_end, on='DOI',how='inner')
DOI_auth_count_end.shape

(783, 68)

In [73]:
Author_stats_table = pd.DataFrame([DOI_auth_count.auth_number.describe(),
                                   DOI_auth_count_start.auth_number.describe(),
                                   DOI_auth_count_mid.auth_number.describe(),
                                   DOI_auth_count_end.auth_number.describe()], index=['All','Start', 'Mid', 'End']).round(2)

Author_stats_table#.to_clipboard() 

,count,mean,std,min,25%,50%,75%,max
All,2347.0,17.99,65.46,1.0,5.0,9.0,14.0,2467.0
Start,774.0,14.15,26.59,1.0,4.0,8.0,13.0,322.0
Mid,779.0,17.42,41.59,1.0,5.0,8.0,14.0,496.0
End,777.0,20.87,98.21,1.0,5.0,9.0,15.0,2467.0


In [74]:
DOI_auth_count.sort_index(inplace=True)
DOI_auth_count['expanding_mean']= DOI_auth_count.auth_number.expanding().mean()

In [75]:
author_numb_figure =(DOI_auth_count.hvplot.scatter(x='created.date-time',y='auth_number',
                                                   xticks =None,logy=True, yaxis='left', alpha=0.2)\
*DOI_auth_count.hvplot.line(x='created.date-time',y='expanding_mean', yaxis='right', color='Red', line_width=4,xticks =None))

author_numb_figure.opts(width=800, height=500, xlabel='Date', ylabel='Number of Authors (log10)')

:Overlay
   .Scatter.I :Scatter   [created.date-time]   (auth_number)
   .Curve.I   :Curve   [created.date-time]   (expanding_mean)

In [76]:
max_authors = DOI_auth_count.reset_index(drop=True)


In [77]:
max_authors.iloc[max_authors.auth_number.idxmax()]['title']

['Guidelines for the use and interpretation of assays for monitoring autophagy (3rd edition)']

In [78]:
DOI_auth_count[DOI_auth_count.auth_number.isna()]

,reference-count,publisher,issue,license,short-container-title,DOI,type,page,source,is-referenced-by-count,...,update-to,event.name,event.location,relation.has-preprint,event.start.date-parts,event.end.date-parts,relation.has-review,auth_number,expanding_median,expanding_mean
created.date-time,,,,,,,,,,,,,,,,,,,,,
2012-02-29 22:01:11+00:00,0,Massachusetts Medical Society,9,NaN,[N Engl J Med],10.1056/nejmx120009,journal-article,868-868,Crossref,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.815789
2012-05-30 15:47:05+00:00,35,Elsevier BV,9834,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[The Lancet],10.1016/s0140-6736(12)60768-5,journal-article,2352-2363,Crossref,592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.443478
2013-05-07 08:39:30+00:00,1,Ovid Technologies (Wolters Kluwer Health),3,NaN,[Medicine],10.1097/md.0b013e3182963750,journal-article,190,Crossref,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.058116
2013-05-22 22:12:03+00:00,5,Massachusetts Medical Society,21,NaN,[N Engl J Med],10.1056/nejmc1303486,journal-article,2036-2037,Crossref,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.885057
2013-07-29 14:53:16+00:00,0,BMJ,jul29 2,NaN,[BMJ],10.1136/bmj.f4799,journal-article,f4799-f4799,Crossref,0,...,"[{'updated': {'date-parts': [[2013, 7, 16]], '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.600318
2013-08-23 09:11:42+00:00,1,Springer Nature,4,"[{'URL': 'http://www.springer.com/tdm', 'start...",[Br Dent J],10.1038/sj.bdj.2013.807,journal-article,173-173,Crossref,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.711916
2014-02-12 17:09:47+00:00,1,Wiley,1,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,[Ann Neurol.],10.1002/ana.24105,journal-article,166-167,Crossref,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.932147
2014-03-05 22:01:27+00:00,8,Massachusetts Medical Society,10,NaN,[N Engl J Med],10.1056/nejmc1315848,journal-article,976-978,Crossref,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.993506
2015-03-13 06:46:14+00:00,1,Springer Nature,5,"[{'URL': 'http://www.springer.com/tdm', 'start...",[Br Dent J],10.1038/sj.bdj.2015.166,journal-article,291-291,Crossref,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.948754


In [87]:
df1.merge(records_df, left_on=[0], right_on='DOI', how='outer',indicator='match' )

,0,reference-count,publisher,issue,license,short-container-title,DOI,type,page,source,...,relation.is-supplemented-by,assertion,update-to,event.name,event.location,relation.has-preprint,event.start.date-parts,event.end.date-parts,relation.has-review,match
0,10.1111/j.1460-9592.2011.03591.x,48.0,Wiley,5,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,[],10.1111/j.1460-9592.2011.03591.x,journal-article,615-622,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,10.1016/j.neurobiolaging.2011.05.018,55.0,Elsevier BV,4,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[Neurobiology of Aging],10.1016/j.neurobiolaging.2011.05.018,journal-article,825.e25-825.e36,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,10.1037/a0024992,0.0,American Psychological Association (APA),4,NaN,[Emotion],10.1037/a0024992,journal-article,778-784,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,10.1016/j.neuroimage.2011.09.010,20.0,Elsevier BV,2,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[NeuroImage],10.1016/j.neuroimage.2011.09.010,journal-article,648-651,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,10.1002/hbm.21402,41.0,Wiley,12,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,[Hum. Brain Mapp.],10.1002/hbm.21402,journal-article,2802-2814,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,10.1212/wnl.0000000000001606,0.0,Ovid Technologies (Wolters Kluwer Health),21,NaN,[Neurology],10.1212/wnl.0000000000001606,journal-article,2132-2145,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
2379,10.1016/s0140-6736(15)60692-4,117.0,Elsevier BV,9995,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[The Lancet],10.1016/s0140-6736(15)60692-4,journal-article,743-800,Crossref,...,NaN,"[{'value': 'Elsevier', 'name': 'publisher', 'l...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
2380,10.1016/b978-0-444-63432-0.00009-8,92.0,Elsevier,NaN,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[],10.1016/b978-0-444-63432-0.00009-8,book-chapter,147-163,Crossref,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
2381,10.1016/s2352-3018(16)30087-x,41.0,Elsevier BV,8,[{'URL': 'https://www.elsevier.com/tdm/userlic...,[The Lancet HIV],10.1016/s2352-3018(16)30087-x,journal-article,e361-e387,Crossref,...,NaN,"[{'value': 'Elsevier', 'name': 'publisher', 'l...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
